In [ ]:
import requests
import pandas as pd
import numpy as np
import sweetviz as sv
import os
import shutil

def request(url, params):
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error en la solicitud: {response.status_code}")
        return None

url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
params = {
    'format': 'geojson',
    'starttime': '2014-10-21',
    'endtime': '2015-10-21',
    'minmagnitude': 0,
    'limit': 20000,
}

response = request(url, params)

In [ ]:
if response is not None:
    earthquakes = []
    for feature in response['features']:
        properties = feature['properties']
        geometry = feature['geometry']

        earthquake_info = {
            'place': properties['place'],
            'magnitude': properties['mag'],
            'time': pd.to_datetime(properties['time'], unit='ms'),
            'latitude': geometry['coordinates'][1],
            'longitude': geometry['coordinates'][0],
            'depth': geometry['coordinates'][2],
        }
        earthquakes.append(earthquake_info)

    df = pd.DataFrame(earthquakes)

    csv_path = '../results/earthquakes_last_10_years.csv'
    df.to_csv(csv_path, index=False)
    print(f"Datos guardados en '{csv_path}'.")

    excel_path = '../results/earthquakes_last_10_years.xlsx'
    df.to_excel(excel_path, index=False)
    print(f"Datos guardados en '{excel_path}'.")

    report = sv.analyze(df)  
    report.show_html('earthquake_data_report.html')  
    print("Análisis exploratorio de datos guardado en 'earthquake_data_report.html'.")

In [ ]:
from pycaret.regression import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sweetviz as sv
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import gradio as gr
warnings.filterwarnings('ignore')

def preprocess_data(df):
    try:
        df = df.copy()
        df = df.drop(columns=['place', 'time'])
        return df
    except Exception as e:
        print(f"Error in data preprocessing: {str(e)}")
        raise

def create_visualization(df, plot_type, save_path=None):
    try:
        plt.figure(figsize=(12, 8))

        if plot_type == 'correlation':
            sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
            plt.title('Correlation Matrix of Features')

        elif plot_type == 'magnitude_dist':
            sns.histplot(df['magnitude'], kde=True, color='b', bins=30)
            plt.title('Earthquake Magnitude Distribution')
            plt.xlabel('Magnitude')
            plt.ylabel('Frequency')

        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.show()
        plt.close()

    except Exception as e:
        print(f"Error in visualization creation: {str(e)}")
        plt.close()

def train_earthquake_model(df, test_size=0.2, random_state=42):
    try:
        X = df.drop(columns=['magnitude'])
        y = df['magnitude']

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )

        train_data = pd.concat([X_train, y_train], axis=1)
        test_data = pd.concat([X_test, y_test], axis=1)

        exp_reg = setup(
            data=train_data,
            target='magnitude',
            session_id=random_state,
            normalize=True,
            transformation=True,
            numeric_features=['latitude', 'longitude', 'depth'],
            verbose=False
        )

        best_model = compare_models(n_select=1)
        final_model = create_model(best_model)

        predictions = predict_model(final_model, data=test_data)

        plot_model(final_model, plot='residuals', save=True)
        plot_model(final_model, plot='error', save=True)

        final_tuned_model = finalize_model(final_model)
        save_model(final_tuned_model, 'earthquake_model_v1')

        return final_tuned_model, predictions

    except Exception as e:
        print(f"Error in model training: {str(e)}")
        raise

def main():
    try:
        processed_df = preprocess_data(df)

        create_visualization(processed_df, 'correlation', 'correlation_matrix.png')
        create_visualization(processed_df, 'magnitude_dist', 'magnitude_distribution.png')

        model, predictions = train_earthquake_model(processed_df)

        train_size = 0.8
        train_df = processed_df.sample(frac=train_size, random_state=42)
        test_df = processed_df.drop(train_df.index)

        report = sv.compare([train_df, "Training"], [test_df, "Test"],
                            target_feat="magnitude")
        report.show_html('earthquake_analysis_report.html')

        print("Analysis completed successfully!")
        print(f"Model performance metrics:")
        print(predictions[['magnitude', 'prediction_label']].describe())

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

def predict_magnitude(latitude, longitude, depth):
    try:
        data = {
            'latitude': [latitude],
            'longitude': [longitude],
            'depth': [depth]
        }

        input_data = pd.DataFrame(data)

        model = load_model('earthquake_model_v1')
        prediction = predict_model(model, data=input_data)
        return float(prediction['prediction_label'].values[0])

    except Exception as e:
        print(f"Error in magnitude prediction: {str(e)}")
        return f"Prediction Error: {str(e)}"

if __name__ == '__main__':
    main()

    # Interfaz Gradio
    inputs = [
        gr.Number(label="Latitude"),
        gr.Number(label="Longitude"),
        gr.Number(label="Depth (km)")
    ]

    output = gr.Number(label="Predicted Magnitude")

    interface = gr.Interface(
        fn=predict_magnitude,
        inputs=inputs,
        outputs=output,
        title="Earthquake Magnitude Prediction",
        description="Enter earthquake parameters to predict the magnitude."
    )

    interface.launch()

In [ ]:
create_visualization(df, 'correlation', 'correlation_matrix.png')

In [ ]:
create_visualization(df, 'magnitude_dist', 'magnitude_distribution.png')

In [ ]:
create_visualization(df, 'yearly_count', 'yearly_frequency.png')